# Notebook Setup

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filename in os.walk('/kaggle/input/financial-news-headlines/reuters_headlines.csv'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/financial-news-headlines/guardian_headlines.csv
/kaggle/input/financial-news-headlines/cnbc_headlines.csv
/kaggle/input/financial-news-headlines/reuters_headlines.csv


In [1]:
from huggingface_hub import notebook_login

notebook_login()

In [2]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
wandb_api_key=user_secrets.get_secret("WANDB_API_KEY")

In [3]:
import os
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
os.environ["WANDB_API_KEY"] =user_secrets.get_secret("WANDB_API_KEY")
os.environ["WANDB_PROJECT"] = "T5-FNHR_Entitle"

In [4]:
!wandb login

wandb: Currently logged in as: ankonbh (ankonbh-university-of-leeds) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [5]:
import numpy as np 
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from datasets import Dataset,load_from_disk,load_dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
#from peft import LoraConfig, get_peft_model, TaskType
import torch

2025-07-23 16:29:57.166512: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753288197.534190      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753288197.640891      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Data Understanding and Preparation

In [18]:
df=pd.read_csv('/kaggle/input/financial-news-headlines/reuters_headlines.csv')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32770 entries, 0 to 32769
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Headlines    32770 non-null  object
 1   Time         32770 non-null  object
 2   Description  32770 non-null  object
dtypes: object(3)
memory usage: 768.2+ KB


In [5]:
df.head()

,Headlines,Time,Description
0,TikTok considers London and other locations fo...,Jul 18 2020,TikTok has been in discussions with the UK gov...
1,Disney cuts ad spending on Facebook amid growi...,Jul 18 2020,Walt Disney has become the latest company to ...
2,Trail of missing Wirecard executive leads to B...,Jul 18 2020,Former Wirecard chief operating officer Jan M...
3,Twitter says attackers downloaded data from up...,Jul 18 2020,Twitter Inc said on Saturday that hackers were...
4,U.S. Republicans seek liability protections as...,Jul 17 2020,A battle in the U.S. Congress over a new coron...


In [6]:
df.isnull().sum()

Headlines      0
Time           0
Description    0
dtype: int64

In [19]:
df_original=df.copy()

In [20]:
df=df_original.loc[:,df_original.columns[[0,2]]]

In [11]:
df_word_count=pd.DataFrame()
for x in df.columns:
    df_word_count[x]=df[x].apply(lambda x: len(word_tokenize(x)))

In [12]:
df_word_count.describe()

,Headlines,Description
count,32770.000000,32770.000000
mean,11.289136,37.694599
std,2.279351,7.293127
min,3.000000,6.000000
25%,10.000000,33.000000
50%,11.000000,38.000000
75%,13.000000,42.000000
max,22.000000,99.000000


In [21]:
df.iloc[:,1]=df.iloc[:,1].apply(lambda x: f'entitle: {x} ')

In [23]:
df.head()

,Headlines,Description
0,TikTok considers London and other locations fo...,entitle: TikTok has been in discussions with t...
1,Disney cuts ad spending on Facebook amid growi...,entitle: Walt Disney has become the latest co...
2,Trail of missing Wirecard executive leads to B...,entitle: Former Wirecard chief operating offi...
3,Twitter says attackers downloaded data from up...,entitle: Twitter Inc said on Saturday that hac...
4,U.S. Republicans seek liability protections as...,entitle: A battle in the U.S. Congress over a ...


## HuggingFace Dataset

In [32]:
hf_dataset = Dataset.from_pandas(df)

In [33]:
hf_dataset=hf_dataset.train_test_split(test_size=0.25, seed=42)
hf_dataset_train=hf_dataset['train']
hf_dataset_test=hf_dataset['test']

In [34]:
hf_dataset_train=hf_dataset_train.train_test_split(test_size=0.2, seed=42)
hf_dataset_val=hf_dataset_train['test']
hf_dataset_train=hf_dataset_train['train']

In [35]:
from datasets import concatenate_datasets,DatasetDict
tokenized_dataset = DatasetDict({
    "train":hf_dataset_train,
    "val":hf_dataset_val,
    "test":hf_dataset_test,})

In [36]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['Headlines', 'Description'],
        num_rows: 19661
    })
    val: Dataset({
        features: ['Headlines', 'Description'],
        num_rows: 4916
    })
    test: Dataset({
        features: ['Headlines', 'Description'],
        num_rows: 8193
    })
})

## Tokenization

In [6]:
model_name="google/flan-t5-base"

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

In [8]:
max_input_length = 512
max_target_length = 128

def preprocess_function(examples):
    inputs = [doc for doc in examples["Description"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    labels = tokenizer(text_target=examples["Headlines"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [41]:
tokenized_dataset = tokenized_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/19661 [00:00<?, ? examples/s]

Map:   0%|          | 0/4916 [00:00<?, ? examples/s]

Map:   0%|          | 0/8193 [00:00<?, ? examples/s]

In [45]:
tokenized_dataset.push_to_hub("Ankonbh/Financial-News-Headlines-Reuters")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/20 [00:00<?, ?ba/s]

Uploading...:   0%|          | 0.00/6.64M [00:00<?, ?B/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/5 [00:00<?, ?ba/s]

Uploading...:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/9 [00:00<?, ?ba/s]

Uploading...:   0%|          | 0.00/2.77M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/Ankonbh/Financial-News-Headlines-Reuters/commit/8d938a5d9e73fa401c61510a1aa4fdc79111e959', commit_message='Upload dataset', commit_description='', oid='8d938a5d9e73fa401c61510a1aa4fdc79111e959', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/Ankonbh/Financial-News-Headlines-Reuters', endpoint='https://huggingface.co', repo_type='dataset', repo_id='Ankonbh/Financial-News-Headlines-Reuters'), pr_revision=None, pr_num=None)

# Modelling

In [6]:
model_name="google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

In [7]:
tokenized_dataset=load_dataset("Ankonbh/Financial-News-Headlines-Reuters")

README.md:   0%|          | 0.00/639 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/6.64M [00:00<?, ?B/s]

data/val-00000-of-00001.parquet:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/2.77M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/19661 [00:00<?, ? examples/s]

Generating val split:   0%|          | 0/4916 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/8193 [00:00<?, ? examples/s]

In [8]:
#model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
model=AutoModelForSeq2SeqLM.from_pretrained("Ankonbh/flan-t5-base-finetuned-FNHR")

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

In [9]:
batch_size = 16
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-FNHR",
    eval_strategy = "epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
    report_to='wandb',
)

In [10]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [11]:
!pip install evaluate rouge_score

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 7.2 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=a557070f5122b77b04608d6af028adf3ebf60097764e72ff338f83b70638d63c
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.5.1
    Uninstalling fsspec-2025.5.1:
      Successfully uninstalled fsspec-2025.5.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.8.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which 

In [12]:
from evaluate import load
metric = load("rouge")

In [13]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.where(predictions != -100, predictions, tokenizer.pad_token_id)
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    # Note that other metrics may not have a `use_aggregator` parameter
    # and thus will return a list, computing a metric for each sentence.
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True, use_aggregator=True)
    # Extract a few results
    result = {key: value * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    return {k: round(v, 4) for k, v in result.items()}

In [14]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["val"],
    data_collator=data_collator,
    processing_class=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

In [15]:
prediction_output = trainer.predict(tokenized_dataset['test'])

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: ankonbh (ankonbh-university-of-leeds) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [21]:
prediction_output.metrics

{'test_loss': 1.8388476371765137,
 'test_rouge1': 46.9203,
 'test_rouge2': 22.3549,
 'test_rougeL': 42.7079,
 'test_rougeLsum': 42.7135,
 'test_gen_len': 15.9284,
 'test_runtime': 221.1821,
 'test_samples_per_second': 37.042,
 'test_steps_per_second': 1.162}

In [19]:
model_pt = AutoModelForSeq2SeqLM.from_pretrained(model_name)

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [20]:
batch_size = 16
args_pt = Seq2SeqTrainingArguments(
    f"{model_name}-pretrained-FNHR",
    eval_strategy = "epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=False,
    report_to='wandb',
)

trainer_pt = Seq2SeqTrainer(
    model_pt,
    args_pt,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["val"],
    data_collator=data_collator,
    processing_class=tokenizer,
    compute_metrics=compute_metrics
)

In [22]:
prediction_output_pt = trainer_pt.predict(tokenized_dataset['test'])

/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


In [25]:
prediction_output_pt.metrics

{'test_loss': 2.253880739212036,
 'test_rouge1': 39.1419,
 'test_rouge2': 16.754,
 'test_rougeL': 35.6027,
 'test_rougeLsum': 35.6048,
 'test_gen_len': 14.7954,
 'test_runtime': 222.9179,
 'test_samples_per_second': 36.753,
 'test_steps_per_second': 1.153}

In [42]:
data=[prediction_output_pt.metrics,prediction_output.metrics]
df_performance=pd.DataFrame(data)

In [43]:
df_performance

,test_loss,test_rouge1,test_rouge2,test_rougeL,test_rougeLsum,test_gen_len,test_runtime,test_samples_per_second,test_steps_per_second
0,2.253881,39.1419,16.7540,35.6027,35.6048,14.7954,222.9179,36.753,1.153
1,1.838848,46.9203,22.3549,42.7079,42.7135,15.9284,221.1821,37.042,1.162


In [58]:
((df_performance.iloc[1]-df_performance.iloc[0]))

test_loss                 -0.415033
test_rouge1                7.778400
test_rouge2                5.600900
test_rougeL                7.105200
test_rougeLsum             7.108700
test_gen_len               1.133000
test_runtime              -1.735800
test_samples_per_second    0.289000
test_steps_per_second      0.009000
dtype: float64

In [57]:
((df_performance.iloc[1]-df_performance.iloc[0])*100/df_performance.iloc[0])

test_loss                 -18.414155
test_rouge1                19.872311
test_rouge2                33.430226
test_rougeL                19.956913
test_rougeLsum             19.965566
test_gen_len                7.657786
test_runtime               -0.778672
test_samples_per_second     0.786330
test_steps_per_second       0.780572
dtype: float64

In [51]:
df_perc

,0
test_loss,-18.414155
test_rouge1,19.872311
test_rouge2,33.430226
test_rougeL,19.956913
test_rougeLsum,19.965566
test_gen_len,7.657786
test_runtime,-0.778672
test_samples_per_second,0.786330
test_steps_per_second,0.780572


In [50]:
pd.concat([df_performance,df_perc])

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,test_loss,test_rouge1,test_rouge2,test_rougeL,test_rougeLsum,test_gen_len,test_runtime,test_samples_per_second,test_steps_per_second,0
0,2.253881,39.1419,16.7540,35.6027,35.6048,14.7954,222.9179,36.753,1.153,NaN
1,1.838848,46.9203,22.3549,42.7079,42.7135,15.9284,221.1821,37.042,1.162,NaN
test_loss,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-18.414155
test_rouge1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.872311
test_rouge2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33.430226
test_rougeL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.956913
test_rougeLsum,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.965566
test_gen_len,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.657786
test_runtime,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.778672
test_samples_per_second,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.786330


In [39]:
df_performance

,test_loss,test_rouge1,test_rouge2,test_rougeL,test_rougeLsum,test_gen_len,test_runtime,test_samples_per_second,test_steps_per_second
0,2.253881,39.1419,16.7540,35.6027,35.6048,14.7954,222.9179,36.753,1.153
1,1.838848,46.9203,22.3549,42.7079,42.7135,15.9284,221.1821,37.042,1.162
